### Can we classify each phase as relatively low or high anxiety for each subject? ###
#### APD, WESAD ####

In [23]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_wesad as dr_w
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [32]:
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
]

# model_phases_apd = [
#     [
#         "Baseline_Rest", 
#         "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
#         "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
#     ]
# ]

model_phases_apd = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

model_phases_wesad = [
    [
        dr_w.Phases.BASE,
        dr_w.Phases.FUN,
        dr_w.Phases.TSST,
        dr_w.Phases.MEDI_1,
        dr_w.Phases.MEDI_2
    ]
]

# model_phases_wesad = [
#     dr_w.Phases.BASE,
#     dr_w.Phases.FUN,
#     dr_w.Phases.TSST,
#     dr_w.Phases.MEDI_1,
#     dr_w.Phases.MEDI_2
# ]

# anxiety_label_type = "Anxiety"
anxiety_label_type = None
wesad_label_type = "stai"

models = {
    "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    # "KNN": KNeighborsClassifier(n_neighbors=7),
    # "DT": DecisionTreeClassifier(),
    "LogReg": LogisticRegression(max_iter=1000),
    # "Bayes": GaussianNB(),
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="logloss")
}

threshold = "fixed"
test_size = 1.0

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr_a.Phases.PHASES_LIST)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

In [31]:
# TRAIN ON APD AND TEST ON WESAD
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


for i, phases_apd in enumerate(model_phases_apd):
    for j, phases_wesad in enumerate(model_phases_wesad):
        phases_wesad = [phases_wesad]
        print(f"APD PHASES {i} " + "-"*50)
        print(f"WESAD PHASES {phases_wesad} " + "-"*50)
        x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold)
        x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold)
        # drop subjects with noisy data
        x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)]
        y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)]
        
        if anxiety_label_type is not None:
            x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature
        
        # x = x[x['subject'] != 8.0]
        # y = y[y['subject'] != 8.0]
        
        x_a = x_a.drop(["phaseId"], axis=1)
        x_b = x_b.drop(["phaseId"], axis=1)

        # make sure subjects from different datasets aren't labeled with the same index
        x_b["subject"] = x_b["subject"] + 500
        y_b["subject"] = y_b["subject"] + 500

        # print(f"y_a:\n{y_a.loc[:, 'label'].value_counts()}")
        # print(f"y_b:\n{y_b.loc[:, 'label'].value_counts()}")

        # 0-1 scaling
        for c in range(3, len(x_a.columns)):
            data_col = x_a[x_a.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_a[x_a.columns[c]] = data_col
        # 0-1 scaling
        for c in range(3, len(x_b.columns)):
            data_col = x_b[x_b.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_b[x_b.columns[c]] = data_col

        acc_results = {
            "SVM": [],
            "LogReg": [],
            "XGB": []
        }
        reports = {
            "SVM": [],
            "LogReg": [],
            "XGB": []
        }
        num_iters = 1
        for _ in range(num_iters):
            out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size)
            for model_name in acc_results:
                acc_results[model_name].append(out[model_name][0])
                reports[model_name].append(out[model_name][1])

        for model_name in acc_results.keys():
            acc = np.mean(acc_results[model_name])
            print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
            if acc > 0.55:
                print(f"Model evaluation metrics for {model_name}:")
                p = np.mean([report["precision"] for report in reports[model_name]])
                r = np.mean([report["recall"] for report in reports[model_name]])
                f1 = np.mean([report["f1"] for report in reports[model_name]])
                auc = np.mean([report["auc"] for report in reports[model_name]])
                report = reports[model_name]
                print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
        print("\n")

APD PHASES 0 --------------------------------------------------
WESAD PHASES ['Base'] --------------------------------------------------
y_train:
1    253
0     71
Name: label, dtype: int64
y_test:
0    14
Name: label, dtype: int64
Model SVM, Predictions: [1], [14]
Only one class present in y_true. ROC AUC score is not defined in that case. Setting AUC score to -1.
Model LogReg, Predictions: [1], [14]
Only one class present in y_true. ROC AUC score is not defined in that case. Setting AUC score to -1.
Model XGB, Predictions: [1], [14]
Only one class present in y_true. ROC AUC score is not defined in that case. Setting AUC score to -1.
y_train:
1    253
0     71
Name: label, dtype: int64
y_test:
0    14
Name: label, dtype: int64
Model SVM, Predictions: [1], [14]
Only one class present in y_true. ROC AUC score is not defined in that case. Setting AUC score to -1.
Model LogReg, Predictions: [1], [14]
Only one class present in y_true. ROC AUC score is not defined in that case. Setting AUC 

In [38]:
# TRAIN ON WESAD AND TEST ON APD
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


for i, phases_wesad in enumerate(model_phases_wesad):
    for j, phases_apd in enumerate(model_phases_apd):
        print(f"WESAD PHASES {i} " + "-"*50)
        print(f"APD PHASES {phases_apd} " + "-"*50)
        phases_apd = [phases_apd]
        x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold)
        x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold)
        print(y_b)
        # drop subjects with noisy data
        x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)]
        y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)]
        if anxiety_label_type is not None:
            x_b = x_b.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature
        # x = x[x['subject'] != 8.0]
        # y = y[y['subject'] != 8.0]
        
        x_a = x_a.drop(["phaseId"], axis=1)
        x_b = x_b.drop(["phaseId"], axis=1)

        # make sure subjects from different datasets aren't labeled with the same index
        x_b["subject"] = x_b["subject"] + 500
        y_b["subject"] = y_b["subject"] + 500

        # 0-1 scaling
        for c in range(3, len(x_a.columns)):
            data_col = x_a[x_a.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_a[x_a.columns[c]] = data_col
        # 0-1 scaling
        for c in range(3, len(x_b.columns)):
            data_col = x_b[x_b.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_b[x_b.columns[c]] = data_col

        print(f"y_a:\n{y_a.loc[:, 'label'].value_counts()}")
        print(f"y_b:\n{y_b.loc[:, 'label'].value_counts()}")

        acc_results = {
        "SVM": [],
        "LogReg": [],
        "XGB": []
        }
        reports = {
        "SVM": [],
        "LogReg": [],
        "XGB": []
        }
        num_iters = 1
        for _ in range(num_iters):
            out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size)
            for model_name in acc_results:
                acc_results[model_name].append(out[model_name][0])
                reports[model_name].append(out[model_name][1])

        for model_name in acc_results.keys():
            acc = np.mean(acc_results[model_name])
            print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
            if acc > 0.55:
                p = np.mean([report["precision"] for report in reports[model_name]])
                r = np.mean([report["recall"] for report in reports[model_name]])
                f1 = np.mean([report["f1"] for report in reports[model_name]])
                auc = np.mean([report["auc"] for report in reports[model_name]])
                report = reports[model_name]
                print(f"Report:\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}")
        print("\n")

WESAD PHASES 0 --------------------------------------------------
APD PHASES Baseline_Rest --------------------------------------------------
    subject  label
0      14.0      0
1      21.0      1
2      23.0      0
3      25.0      0
4      34.0      1
5      39.0      1
6      43.0      1
7      46.0      1
8      51.0      1
9      57.0      1
10     71.0      1
11     72.0      1
12     77.0      1
13     78.0      1
14     79.0      1
15     80.0      1
16     82.0      1
17     83.0      1
18     84.0      1
19     85.0      1
20     87.0      1
21     88.0      1
22     89.0      1
23     91.0      1
24     92.0      1
25     93.0      1
26     10.0      0
27     12.0      1
28     15.0      0
29     16.0      0
30     18.0      0
31     22.0      0
32     26.0      0
33     27.0      1
34     29.0      1
35     31.0      1
36     32.0      1
37     33.0      1
38     35.0      1
39      4.0      0
40     42.0      1
41     45.0      1
42     47.0      1
43     48.0      1
44 